# FLEE-GNN - Predicting network resilience via GNN and cross-silo federated learning

## load libraries

In [1]:
from collections import OrderedDict
from typing import List, Tuple, Union, Dict, Optional

import pandas as pd
import numpy as np
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import json
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
import torch.optim as optim
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch_geometric.nn import MessagePassing
import torch.nn.init as init

import argparse
import os

import flwr as fl

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Metrics,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)

from flwr.server.client_proxy import ClientProxy

/home/jinmeng/ana3/envs/neo4j/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## define network structure and related functions

In [2]:
class GNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, edge_dim):
        super(GNNLayer, self).__init__(aggr='add')
        self.lin = nn.Linear(in_channels + edge_dim, out_channels)
        init.kaiming_normal_(self.lin.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.edge_dim = edge_dim

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_index, edge_attr):
        node_edge_feature = torch.cat([x_j, edge_attr], dim=-1)
        return F.leaky_relu(self.lin(node_edge_feature), negative_slope=0.01)

class GNN(torch.nn.Module):
    def __init__(self, node_in_channels, edge_in_channels, hidden_channels):
        super(GNN, self).__init__()
        self.conv1 = GNNLayer(node_in_channels, hidden_channels, edge_in_channels)
        self.conv2 = GNNLayer(hidden_channels, hidden_channels * 2, edge_in_channels)
        self.conv3 = GNNLayer(hidden_channels * 2, hidden_channels * 4, edge_in_channels)
        self.fc1 = nn.Linear(hidden_channels * 2, hidden_channels)
        init.kaiming_normal_(self.fc1.weight, mode='fan_in', nonlinearity='leaky_relu')
        self.fc2 = nn.Linear(hidden_channels, 1)
        init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='linear')
        self.sigmoid = nn.Sigmoid()

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.leaky_relu(x, negative_slope=0.01)
        x = self.fc2(x)
        x = self.sigmoid(x)   
        return x
    
def create_edge_attr_index(df):
    df1 = df[["COMM_TTL", "VAL", "TON", "AVGMILE", "Origin", "Destination"]]
    # First, create a MultiIndex for the new columns using 'Commodity Category'
    df1 = df1.set_index(["COMM_TTL", 'Origin', 'Destination'])

    # Unstack the MultiIndex to create a wide format dataframe
    df2 = df1.unstack(level="COMM_TTL")
    # Flatten the MultiIndex columns

    # Reset the index to obtain 'Origin' and 'Destination' as columns
    df2.reset_index(inplace=True)

    df2.sort_index(axis=1, inplace=True)

    # Fill NaN values with 0
    df2.fillna(0, inplace=True)

     # Ensure all columns for 'COMM_TTL' are present
    for i in range(1, 9):
        for col_name in ['VAL', 'TON', 'AVGMILE']:
            if (col_name, i) not in df2.columns:
                df2[(col_name, i)] = 1.0

    # Create a separate dataframe for edge_attr without 'Origin' and 'Destination' columns
    edge_attr = df2.drop(columns=['Origin', 'Destination'])
    edge_attr.fillna(0.0, inplace=True)

    # Create a separate dataframe for edge_attr with 'Origin' and 'Destination' columns
    edge_index = df2[['Origin', 'Destination']]

    return edge_attr, edge_index

## define data path and data preprocessing

In [3]:
info_dir = './data/info/'

# abreviation to full name
with open(os.path.join(info_dir, 'state_a2f.json'), 'r') as f:
    state_a2f_dict = json.load(f)
    
# full name to abreviation
state_f2a_dict = {}
for key in state_a2f_dict:
    state_f2a_dict[state_a2f_dict[key]] = key
    
# state full name to latitute and longitude
with open(os.path.join(info_dir, 'state_f2latlon.json'), 'r') as f:
    state_f2latlon_list = json.load(f)
    
# state abreviation to latitute and longitude
state_a2latlon_dict = {}
for d in state_f2latlon_list:
    state_a2latlon_dict[state_f2a_dict[d['state']]] = {'lat': d["latitude"], 'lon': d["longitude"]}

In [4]:
# note that here we train models on generated data with noise level of 0.2

whole_data_dir = './training/noise_data_0.2/'
whole_label_dir = './training/noise_label_0.2/'
whole_output_dir = './training/noise_output_0.2/'

data_region_WEST_dir = './training/noise_data_region_WEST_v2_0.2/'
data_region_MIDWEST_dir = './training/noise_data_region_MIDWEST_v2_0.2/'
data_region_SOUTH_dir = './training/noise_data_region_SOUTH_v2_0.2/'
data_region_NORTHEAST_dir = './training/noise_data_region_NORTHEAST_v2_0.2/'

label_region_WEST_dir = './training/noise_data_region_WEST_v2_0.2/'
label_region_MIDWEST_dir = './training/noise_data_region_MIDWEST_v2_0.2/'
label_region_SOUTH_dir = './training/noise_data_region_SOUTH_v2_0.2/'
label_region_NORTHEAST_dir = './training/noise_data_region_NORTHEAST_v2_0.2/'

output_region_WEST_dir = './training/noise_data_region_WEST_v2_0.2/'
output_region_MIDWEST_dir = './training/noise_data_region_MIDWEST_v2_0.2/'
output_region_SOUTH_dir = './training/noise_data_region_SOUTH_v2_0.2/'
output_region_NORTHEAST_dir = './training/noise_data_region_NORTHEAST_v2_0.2/'

size = 100
epochs = 200

## define dataset generation function

In [5]:
def generate_dataset(data_dir, label_dir):
    dataset = []
    file_not_exist = []
    for index in range(1, size):
        data_path = os.path.join(data_dir, 'data_{}.csv'.format(index))
        label_path = os.path.join(label_dir, 'label_{}.csv'.format(index))

        if not os.path.exists(data_path) or not os.path.exists(label_path):
            file_not_exist.append(index)
            continue

        raw_data = pd.read_csv(data_path)
        raw_label = pd.read_csv(label_path)
        node2idx = {}
        idx2node = {}
        for i, node in enumerate(raw_label['node']):
            node2idx[node] = i
            idx2node[i] = node
        label = raw_label[['node', 'import_resilience']]
        edge_attr, edge_index =create_edge_attr_index(raw_data)
        
        # edge_attr = normalize_features(edge_attr)
        edge_attr_means = edge_attr.mean(axis=0)
        edge_attr_stds = edge_attr.std(axis=0)
        edge_attr = (edge_attr - edge_attr_means) / edge_attr_stds
        edge_attr.fillna(0.00000, inplace=True)

        edge_index = edge_index.applymap(lambda node: node2idx[node]) 
        node_features = []
        for index in idx2node:
            node = idx2node[index]
            node_features.append([state_a2latlon_dict[node]['lat'], state_a2latlon_dict[node]['lon']])
        node_features = np.array(node_features)
        # node_features = normalize_features(node_features)
        node_features_means = node_features.mean(axis=0)
        node_features_stds = node_features.std(axis=0)
        node_features = (node_features - node_features_means) / node_features_stds
        node_features = np.nan_to_num(node_features, nan=0.00000)

        node_features_tensor = torch.tensor(node_features, dtype=torch.float)
        edge_attr_tensor = torch.tensor(edge_attr.values, dtype=torch.float)
        edge_index_tensor = torch.tensor(edge_index.values.T, dtype=torch.long)
        label = label['import_resilience'].values
        label_tensor = torch.tensor(label, dtype=torch.float)
        data = Data(x=node_features_tensor, edge_index=edge_index_tensor, edge_attr=edge_attr_tensor, y=label_tensor)
        dataset.append(data)

    print('file does not exist: ', file_not_exist)
    return dataset

## generate datasets

In [6]:
dataset_region_WEST = generate_dataset(data_region_WEST_dir, label_region_WEST_dir)
dataset_region_MIDWEST = generate_dataset(data_region_MIDWEST_dir, label_region_MIDWEST_dir)
dataset_region_SOUTH = generate_dataset(data_region_SOUTH_dir, label_region_SOUTH_dir)
dataset_region_NORTHEAST = generate_dataset(data_region_NORTHEAST_dir, label_region_NORTHEAST_dir)

file does not exist:  [46, 48, 83]
file does not exist:  []
file does not exist:  []


/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])
/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])
/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])
/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])
/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(

file does not exist:  []


/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])
/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])
/tmp/ipykernel_45000/3302128996.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  edge_attr = df2.drop(columns=['Origin', 'Destination'])


## split data into training and val sets

In [9]:
# Split the data into training and val sets
train_data_region_WEST, val_data_region_WEST = train_test_split(dataset_region_WEST, test_size=0.1, random_state=42)
train_data_region_MIDWEST, val_data_region_MIDWEST = train_test_split(dataset_region_MIDWEST, test_size=0.1, random_state=42)
train_data_region_SOUTH, val_data_region_SOUTH = train_test_split(dataset_region_SOUTH, test_size=0.1, random_state=42)
train_data_region_NORTHEAST, val_data_region_NORTHEAST = train_test_split(dataset_region_NORTHEAST, test_size=0.1, random_state=42)

In [10]:
len(train_data_region_WEST + train_data_region_MIDWEST + train_data_region_SOUTH + train_data_region_NORTHEAST)

353

In [11]:
len(val_data_region_WEST + val_data_region_MIDWEST + val_data_region_SOUTH + val_data_region_NORTHEAST)

40

In [12]:
# Split each training partition into train DataLoader
trainloaders = []
trainloaders.append(DataLoader(train_data_region_WEST, batch_size=32, shuffle=True))
trainloaders.append(DataLoader(train_data_region_MIDWEST, batch_size=32, shuffle=True))
trainloaders.append(DataLoader(train_data_region_SOUTH, batch_size=32, shuffle=True))
trainloaders.append(DataLoader(train_data_region_NORTHEAST, batch_size=32, shuffle=True))

val_loader = DataLoader(val_data_region_SOUTH + val_data_region_MIDWEST + val_data_region_WEST + val_data_region_NORTHEAST, batch_size=32, shuffle=False)

/home/jinmeng/ana3/envs/neo4j/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## define federated client class and train/test functions

In [15]:
node_in_channels = 2
edge_in_channels = 24
hidden_channels = 128

In [16]:
class FedClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"MSE": float(loss)}

    
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

    
def client_fn(cid: str) -> FedClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = GNN(node_in_channels, edge_in_channels, hidden_channels).to(device)

    # Load data
    trainloader = trainloaders[int(cid)]
    valloader = val_loader

    # Create a single client representing a single organization
    return FedClient(net, trainloader, valloader)


def train(net, trainloader, epochs: int, verbose=True):
    """Train the network on the training set."""
    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.001)
    net.train()
    total_loss = 0
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = net(batch)
            loss = criterion(out.view(-1), batch.y)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()
        epoch_loss /= len(trainloader.dataset)
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = nn.MSELoss(reduction='mean')
    net.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in testloader:
            batch = batch.to(device)
            out = net(batch)
            # print(out)
            loss = criterion(out.view(-1), batch.y)
            total_loss += loss.item()
    avg_loss = total_loss / len(testloader.dataset)
    print(f"val loss {avg_loss}")
    return avg_loss

# Add a function to save the model
def save_model(model, path):
    torch.save(model.state_dict(), path)

## aggregation and model saving strategies

In [21]:
device = torch.device('cpu')
save_path = 'weights/test0707'

if not os.path.exists(save_path):
    os.makedirs(save_path)

class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate model weights using weighted average and store checkpoint"""

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)

        if aggregated_parameters is not None:
            print(f"Saving round {server_round} aggregated_parameters...")

            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)
                
            net = GNN(node_in_channels, edge_in_channels, hidden_channels).to(device)

            # Convert `List[np.ndarray]` to PyTorch`state_dict`
            params_dict = zip(net.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            net.load_state_dict(state_dict, strict=True)

            # Save the model
            torch.save(net.state_dict(), f"{save_path}/fl_model_round_{server_round}.pth")

        return aggregated_parameters, aggregated_metrics

## create strategies and start simulation

In [22]:
# Create FedAvg strategy
strategy = SaveModelStrategy(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=4,
    min_evaluate_clients=4,
    min_available_clients=4
)

client_resources = None

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=4,
    config=fl.server.ServerConfig(num_rounds=30),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-07-07 04:27:44,164 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=30, round_timeout=None)
2023-07-07 04:27:48,760	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-07-07 04:27:50,660 | app.py:180 | Flower VCE: Ray initialized with resources: {'accelerator_type:T': 1.0, 'CPU': 72.0, 'GPU': 1.0, 'memory': 181012413850.0, 'object_store_memory': 81862463078.0, 'node:144.92.235.237': 1.0}
INFO flwr 2023-07-07 04:27:50,661 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-07 04:27:50,662 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-07 04:27:52,420 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-07 04:27:52,421 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-07-07 04:27:52,421 | server.py:101 | FL starting
DEBUG flwr 2023-07-07 04:27:52,422 | server.py:218 | fit_round 1: strategy sampled 4 clients (out of 4)


(launch_and_fit pid=68898) Epoch 1: train loss 0.009682374054126525
(launch_and_fit pid=68898) Epoch 2: train loss 0.00903148999374904
(launch_and_fit pid=68898) Epoch 3: train loss 0.008466083347127678
(launch_and_fit pid=68898) Epoch 4: train loss 0.007876379938607805
(launch_and_fit pid=68898) Epoch 5: train loss 0.007320501328854079
(launch_and_fit pid=68898) Epoch 6: train loss 0.006790102030454057
(launch_and_fit pid=68898) Epoch 7: train loss 0.006300839815246925
(launch_and_fit pid=68898) Epoch 8: train loss 0.005869620134321491
(launch_and_fit pid=68898) Epoch 9: train loss 0.005431899193967326
(launch_and_fit pid=68898) Epoch 10: train loss 0.00507262031014046
(launch_and_fit pid=68898) Epoch 11: train loss 0.00472669353645839
(launch_and_fit pid=68898) Epoch 12: train loss 0.004425488496094607
(launch_and_fit pid=68898) Epoch 13: train loss 0.004139926242694426
(launch_and_fit pid=68898) Epoch 14: train loss 0.003876871607276831
(launch_and_fit pid=68898) Epoch 15: train los

DEBUG flwr 2023-07-07 04:27:58,750 | server.py:232 | fit_round 1 received 4 results and 0 failures
WARNING flwr 2023-07-07 04:27:58,760 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-07-07 04:27:58,770 | server.py:168 | evaluate_round 1: strategy sampled 4 clients (out of 4)


(launch_and_fit pid=68867) Epoch 191: train loss 0.0006162850709443682
(launch_and_fit pid=68867) Epoch 192: train loss 0.0006075994310419211
(launch_and_fit pid=68867) Epoch 193: train loss 0.0006169165913643462
(launch_and_fit pid=68867) Epoch 194: train loss 0.0006109308869986052
(launch_and_fit pid=68867) Epoch 195: train loss 0.0006021819380896815
Saving round 1 aggregated_parameters...
(launch_and_fit pid=68867) Epoch 196: train loss 0.0006020693840940347
(launch_and_fit pid=68867) Epoch 197: train loss 0.0005999780730919892
(launch_and_fit pid=68867) Epoch 198: train loss 0.0006028663450747394
(launch_and_fit pid=68867) Epoch 199: train loss 0.0005898905520358782
(launch_and_fit pid=68867) Epoch 200: train loss 0.0005937744392437881


(raylet) [2023-07-07 04:27:58,748 E 68606 68690] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-07-07_04-27-46_909999_45000 is over 95% full, available space: 7481868288; capacity: 233945788416. Object creation will fail if spilling is required.
DEBUG flwr 2023-07-07 04:27:59,468 | server.py:182 | evaluate_round 1 received 4 results and 0 failures
WARNING flwr 2023-07-07 04:27:59,469 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-07-07 04:27:59,469 | server.py:218 | fit_round 2: strategy sampled 4 clients (out of 4)


...

(launch_and_fit pid=68898) Epoch 191: train loss 0.00014301310450340926
(launch_and_fit pid=68898) Epoch 192: train loss 0.00014333630744577125
(launch_and_fit pid=68898) Epoch 193: train loss 0.00014479001042213332
(launch_and_fit pid=68898) Epoch 194: train loss 0.00014428224602944397
(launch_and_fit pid=68898) Epoch 195: train loss 0.00014288946619947976
(launch_and_fit pid=68898) Epoch 196: train loss 0.0001464691117740749
Saving round 30 aggregated_parameters...
(launch_and_fit pid=68898) Epoch 197: train loss 0.00014589659168562862
(launch_and_fit pid=68898) Epoch 198: train loss 0.00014438591583558683
(launch_and_fit pid=68898) Epoch 199: train loss 0.00014549525182568624
(launch_and_fit pid=68898) Epoch 200: train loss 0.00014352983679999127


DEBUG flwr 2023-07-07 04:30:43,967 | server.py:182 | evaluate_round 30 received 4 results and 0 failures
INFO flwr 2023-07-07 04:30:43,968 | server.py:147 | FL finished in 171.54610042460263
INFO flwr 2023-07-07 04:30:43,969 | app.py:218 | app_fit: losses_distributed [(1, 0.0022299263626337053), (2, 0.0016953103244304658), (3, 0.00148029956035316), (4, 0.0013698728289455175), (5, 0.0013007013592869043), (6, 0.001246835058555007), (7, 0.0012077979277819394), (8, 0.0011705752229318022), (9, 0.0011488685850054026), (10, 0.0011346675921231509), (11, 0.0011214660946279765), (12, 0.0011094693327322602), (13, 0.0011045749532058835), (14, 0.0010989872738718987), (15, 0.001095580984838307), (16, 0.0010923496447503567), (17, 0.001087951473891735), (18, 0.001085617090575397), (19, 0.0010900753317400814), (20, 0.0010954789351671935), (21, 0.0010992314433678984), (22, 0.0011053667636588215), (23, 0.001110243797302246), (24, 0.001109963282942772), (25, 0.0011113626882433892), (26, 0.0011111622676253

(launch_and_evaluate pid=68921) val loss 0.001116984593681991
(launch_and_evaluate pid=68867) val loss 0.001116984593681991
(launch_and_evaluate pid=68898) val loss 0.001116984593681991
(launch_and_evaluate pid=68846) val loss 0.001116984593681991


History (loss, distributed):
	round 1: 0.0022299263626337053
	round 2: 0.0016953103244304658
	round 3: 0.00148029956035316
	round 4: 0.0013698728289455175
	round 5: 0.0013007013592869043
	round 6: 0.001246835058555007
	round 7: 0.0012077979277819394
	round 8: 0.0011705752229318022
	round 9: 0.0011488685850054026
	round 10: 0.0011346675921231509
	round 11: 0.0011214660946279765
	round 12: 0.0011094693327322602
	round 13: 0.0011045749532058835
	round 14: 0.0010989872738718987
	round 15: 0.001095580984838307
	round 16: 0.0010923496447503567
	round 17: 0.001087951473891735
	round 18: 0.001085617090575397
	round 19: 0.0010900753317400814
	round 20: 0.0010954789351671935
	round 21: 0.0010992314433678984
	round 22: 0.0011053667636588215
	round 23: 0.001110243797302246
	round 24: 0.001109963282942772
	round 25: 0.0011113626882433892
	round 26: 0.0011111622676253318
	round 27: 0.0011135125998407603
	round 28: 0.0011163843562826514
	round 29: 0.001115348539315164
	round 30: 0.001116984593681991

(raylet) [2023-07-07 04:30:48,888 E 68606 68690] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-07-07_04-27-46_909999_45000 is over 95% full, available space: 7480422400; capacity: 233945788416. Object creation will fail if spilling is required.
(raylet) [2023-07-07 04:30:58,897 E 68606 68690] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-07-07_04-27-46_909999_45000 is over 95% full, available space: 7480401920; capacity: 233945788416. Object creation will fail if spilling is required.
(raylet) [2023-07-07 04:31:08,907 E 68606 68690] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-07-07_04-27-46_909999_45000 is over 95% full, available space: 7480393728; capacity: 233945788416. Object creation will fail if spilling is required.
(raylet) [2023-07-07 04:31:18,915 E 68606 68690] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-07-07_04-27-46_909999_45000 is over 95% full, available space: 7480393728; capacity: 233945788416. Object creation wil